# Pydantic fundamentals

### Start without pandantic

In [1]:
# defined a class Person (blueprint)
class Person:
    def __init__(self, name, gender, age):
        self.name = name 
        self.gender = gender
        self.age = age 

# instantiated an instance from the Person class
person1 = Person(name = "Kokchun", age = 34, gender = "M")
person1

In [2]:
person1.name, person1.age

('Kokchun', 34)

In [3]:
person2 = Person(name = 3213+3213, gender = True, age ="minus ten")
person2

### Validation of our Person class

In [6]:
class Person:
    def __init__(self, name, gender, age):
        if not isinstance(name, str):
            raise TypeError(f"name must be of type str not {type(name)}")
        self.name = name 
        self.gender = gender
        self.age = age 
try:        
    Person(name = 3.1415, age = 34, gender = "M")
except TypeError as err:
    print(err)


name must be of type str not <class 'float'>


In [8]:
class Person:
    def __init__(self, name, gender, age):
        if not isinstance(name, str):
            raise TypeError(f"name must be of type str not {type(name)}")
        self.name = name 
        self.gender = gender
        self.age = age 
    
    def __repr__(self):
        return f"Person({self.name, self.gender, self.age})"
try:        
    person3 = Person(name = "Ragnar", age = -54, gender = "M")
except TypeError as err:
    print(err)
    
person3


Person(('Ragnar', 'M', -54))